In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

In [2]:
from typing import Optional
from functools import partial

import dspy
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI

from sisyphus.heas.label import label_paras
from sisyphus.chain.paragraph import Paragraph, ParagraphExtend
from sisyphus.chain import Filter, Writer
from sisyphus.strategy.run_strategy import extract_main
from sisyphus.strategy.pydantic_models_general import Processing, Material, MaterialDescriptionBase
from sisyphus.strategy.utils import build_process_agent, build_property_agent, build_result_model_contextualized, get_paras_with_props, get_synthesis_paras
from sisyphus.heas.prompt import *
from sisyphus.utils.helper_functions import get_plain_articledb, get_create_resultdb
from sisyphus.heas.synthesis import get_synthesis_prompt


lm = dspy.LM('openai/gpt-4.1-mini')
dspy.configure(lm=lm)
chat_model = ChatOpenAI(model='gpt-4.1-mini')

class StrengthTestBase(BaseModel):
    """Tensile/Compressive test results"""
    ys: Optional[str] = Field(description="Yield strength with unit")
    uts: Optional[str] = Field(description="Ultimate tensile/compressive strength with unit")
    strain: Optional[str] = Field(description="Fracture strain with unit")
    temperature: Optional[str] = Field(description="Test temperature with unit")
    strain_rate: Optional[str] = Field(description="Strain rate with unit")
    other_test_conditions: str = Field(description="Other test conditions, like in salt, hydrogen charging, etc.")

class PhaseInfo(BaseModel):
    """Phase information"""
    phases: list[str] = Field(description="List of phases present in the material")

strength_agent = build_property_agent(
    EXTRACT_PROPERTY_SYS_GENERIC_PROMPT,
    STRENGTH_PROMPT,
    build_result_model_contextualized('Strength', 'Extract tensile/compressive test information from provided text', StrengthTestBase, MaterialDescriptionBase),
    chat_model
)

phase_agent = build_property_agent(
    EXTRACT_PROPERTY_SYS_GENERIC_PROMPT,
    PHASE_PROMPT,
    build_result_model_contextualized('Phase', 'Extract phase information', PhaseInfo, MaterialDescriptionBase),
    chat_model
)

process_agent = build_process_agent(
    EXTRACT_PROCESS_SYS_GENERIC_PROMPT,
    PROCESS_PROMPT,
    Processing,
    chat_model
)

agents_d = {
    'strength': strength_agent,
    'phase': phase_agent
}

def reconstruct_p(paragraphs):
    p_str = ParagraphExtend.from_paragraphs(get_paras_with_props(paragraphs, 'strength'), type='strength')
    p_phase = ParagraphExtend.from_paragraphs(get_paras_with_props(paragraphs, 'phase'), type='phase')
    p_exp = ParagraphExtend.from_paragraphs(get_synthesis_paras(paragraphs) + get_paras_with_props(paragraphs, 'composition'), type='synthesis')
    return{
        "strength": p_str,
        "phase": p_phase,
        "synthesis": p_exp
    }



/Users/pastalover/miniconda3/envs/sisyphus_context/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
extract = partial(extract_main, reconstruct_paragraph=reconstruct_p, property_agents_d=agents_d, formatted_func=get_synthesis_prompt, synthesis_agent=process_agent, save_to='test_out.json')

db = get_plain_articledb('heas_1531')
getter = Filter(db)

chain = getter + label_paras + extract
chain.compose('10.1002&sol;adem.201900587.html')

TypeError: extract_contextualized_main() missing 1 required positional argument: 'save_to'

In [4]:
lm.inspect_history(10)





[2025-08-21T16:36:27.137320]

System message:

Your input fields are:
1. `paragraph` (str)

Your output fields are:
1. `reasoning` (str)
2. `topic` (Literal[synthesis, characterization, others])

All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## paragraph ## ]]
{paragraph}

[[ ## reasoning ## ]]
{reasoning}

[[ ## topic ## ]]
{topic}        # note: the value you produce must be one of: synthesis; characterization; others

[[ ## completed ## ]]

In adhering to this structure, your objective is: 
        assign topic to paragraphs of HEAs(high entropy alloys) papers. The topics include synthesis, characterization, and others.
        Note: a qualified synthesis paragraph should include the synthesis and processing of materials, including methods such as melting, casting, rolling, annealing, mechnical processes or additive manufacturing. be very strict about your decision.


User message:

[[ ## paragraph ## ]]
The HEA with a nomin

In [4]:
r = _

In [ ]:
r

True

In [13]:
from sisyphus.strategy.contextualized import PaperResult, SafeDumpProcessing

In [36]:
class PaperResult(BaseModel):
    properties: list[StrengthTestBase]
    synthesis: SafeDumpProcessing

    def model_dump(self, **kwargs):
        return {
            "properties": [{prop.__class__.__name__: prop.model_dump(**kwargs)} for prop in self.properties],
            "synthesis": self.synthesis.model_dump(**kwargs)
        }

In [37]:
p = PaperResult(properties=[StrengthTestBase(ys='≈430 MPa', uts='720 MPa', strain='48.1 %', temperature=None, strain_rate=None, other_test_conditions='')], synthesis=SafeDumpProcessing(composition='V10Cr15Mn5Fe35Co10Ni25', steps=[{'induction melting': {'atmosphere': 'vacuum'}}, {'homogenized': {'temperature': 1100, 'duration': '6h', 'atmosphere': 'Ar'}}, {'quenching': {'medium': 'water'}}, {'cold rolled': {'reduction': '79%', 'temperature': 'room temperature'}}, {'annealed': {'temperature': 900, 'duration': '10min', 'atmosphere': ''}}]))

In [38]:
p

PaperResult(properties=[StrengthTestBase(ys='≈430 MPa', uts='720 MPa', strain='48.1 %', temperature=None, strain_rate=None, other_test_conditions='')], synthesis=SafeDumpProcessing(composition='V10Cr15Mn5Fe35Co10Ni25', steps=[{'induction melting': {'atmosphere': 'vacuum'}}, {'homogenized': {'temperature': 1100, 'duration': '6h', 'atmosphere': 'Ar'}}, {'quenching': {'medium': 'water'}}, {'cold rolled': {'reduction': '79%', 'temperature': 'room temperature'}}, {'annealed': {'temperature': 900, 'duration': '10min', 'atmosphere': ''}}]))

In [39]:
print(p.properties[0].model_dump())

{'ys': '≈430 MPa', 'uts': '720 MPa', 'strain': '48.1 %', 'temperature': None, 'strain_rate': None, 'other_test_conditions': ''}


In [40]:
import json
print(json.dumps(p.model_dump(), indent=2, ensure_ascii=False))

{
  "properties": [
    {
      "StrengthTestBase": {
        "ys": "≈430 MPa",
        "uts": "720 MPa",
        "strain": "48.1 %",
        "temperature": null,
        "strain_rate": null,
        "other_test_conditions": ""
      }
    }
  ],
  "synthesis": {
    "composition": "V10Cr15Mn5Fe35Co10Ni25",
    "steps": [
      {
        "induction melting": {
          "atmosphere": "vacuum"
        }
      },
      {
        "homogenized": {
          "temperature": 1100,
          "duration": "6h",
          "atmosphere": "Ar"
        }
      },
      {
        "quenching": {
          "medium": "water"
        }
      },
      {
        "cold rolled": {
          "reduction": "79%",
          "temperature": "room temperature"
        }
      },
      {
        "annealed": {
          "temperature": 900,
          "duration": "10min",
          "atmosphere": ""
        }
      }
    ]
  }
}


In [22]:
print(json.dumps(p.model_dump(mode="json"), indent=2))

{
  "properties": [
    {}
  ],
  "synthesis": {
    "composition": "V10Cr15Mn5Fe35Co10Ni25",
    "steps": [
      {
        "induction melting": {
          "atmosphere": "vacuum"
        }
      },
      {
        "homogenized": {
          "temperature": 1100,
          "duration": "6h",
          "atmosphere": "Ar"
        }
      },
      {
        "quenching": {
          "medium": "water"
        }
      },
      {
        "cold rolled": {
          "reduction": "79%",
          "temperature": "room temperature"
        }
      },
      {
        "annealed": {
          "temperature": 900,
          "duration": "10min",
          "atmosphere": ""
        }
      }
    ]
  }
}
